In [19]:
import matplotlib.pyplot as plt
import seaborn as sns
import math
import matplotlib.colors as mcolors
from matplotlib.patches import Rectangle
from prettytable import PrettyTable
%matplotlib inline
from PIL import Image
import os
import pandas as pd
import numpy as np

from sklearn.preprocessing import OneHotEncoder
import catboost as cb
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler,RobustScaler
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import log_loss
from sklearn.model_selection import GridSearchCV, RepeatedStratifiedKFold

from catboost import Pool
from scipy.stats import uniform
from catboost import CatBoostClassifier
import optuna
from sklearn.metrics import log_loss

In [31]:
def find_col_dtypes(df, target, limit = 3):
    """
    Cette fonction va distinguer les variables numériques et catégorielles selon une limite de valeurs uniques
    Elle affiche aussi le type des variables dans le df inital
    """
    num_cols = df.select_dtypes("number").columns.to_list()
    cat_cols = df.select_dtypes("object").columns.to_list()
    cardinals = [col for col in cat_cols if df[col].nunique() == df.shape[0]]
    cat_but_num = [col for col in cat_cols if (df[col].nunique() > limit) & (col not in cardinals)]
    cat_cols = [col for col in df.columns if (df[col].nunique() < limit) & (col not in [target])]
    num_cols = [col for col in num_cols if (col not in cat_cols) & (col not in [target])] + cat_but_num
   
    print(f"Colonnes numériques : {num_cols}", end="\n\n")
    
    print("Voici le type des colonnes numériques : ", end="\n\n")
    df[num_cols].info()
    
    print(f"Colonnes catégorielles : {cat_cols}", end="\n\n")
    
    print("Voici le type des colonnes catégorielles: ", end="\n\n")
    df[cat_cols].info()    
  
    return num_cols, cat_cols


In [27]:
def exploration(df):
    #Nan et doublons
    print("Ce dataframe comporte ",df.shape[0], "lignes et ",df.shape[1], "colonnes.")
    print("Il comporte ", df.isna().sum().sum(), "lignes comportant des Nan et ", df.duplicated().sum().sum(),"doublons.")
    
    #DEscription features
    print("Voici une description des features : ",end="\n\n") 
    imageLue = Image.open("images\Description_features_detail.png")
    fig = plt.figure(figsize=(14,16))

    table = PrettyTable()
    table.field_names = ["Features","Unique Values"]
    for i in list(df_train.columns) :
        nunique =df_train[str(i)].nunique
        table.add_row([i, f"{nunique()}"])
    print('Unique values in categorical columns : \n')
    print(table)
    plt.imshow(imageLue)



In [28]:
def load_data():
    #chermin vers les données
    path = r"C:\Users\Frederic\Documents\GitHub\Tennis_analysis\Prediction-of-Cirrhosis-Outcomes"
    train_path = os.path.abspath(path) + "/data" +"/train.csv"
    test_path = os.path.abspath(path) + "/data" +"/test.csv"
    
    print("creation de df_train")
    df_train = pd.read_csv(train_path,index_col='id')
    #print(df_train.head())
    print("-->Done",end="\n\n")
    
    print("identification des variables numériques et catégorielles ")
    #identification colonnes numériques et catégorielles
    num_cols, cat_cols = find_col_dtypes(df=df_train, target='Status', limit = 4)
    print("-->Done",end="\n\n")
    
    print("la cible est la colonne :", df_train.columns.get_loc('Status'))
          
    #LabelEncoder pour les variables catégorielles
    print("Colonnes catégorielles : ",cat_cols,end="\n\n")
    
    print("Encodage des variables catégorielles avec Labelencoder")  
    le = LabelEncoder()
    for column in cat_cols:
        df_train[column] = le.fit_transform(df_train[column])
    print("-->Done",end="\n\n")

    # Séparation des fonctionnalités et de la variable cible
    y=df_train['Status']
    #Utilisation de LabelBinarizer pour encoder la variable cible 'Status'
    label_binarizer = LabelBinarizer()
    y_encoded = label_binarizer.fit_transform(y)
    df_train['Status'] = y_encoded

    print("préparation des données")
    #Préparation des données
    target = df_train.Status
    data = df_train.drop(columns=["Status"])
    print("-->Done",end="\n\n")
    
    cat_indicies = []
    for column in cat_cols:
        data[column] = data[column].astype('category')
        cat_indicies.append(data.columns.get_loc(column)) # renvoie les indices de toutes les colonnes non numériques sous forme de liste.
    print("--> les indices des colonnes catégorielles sont : ",cat_indicies)
     
    X_train,X_test,y_train,y_test = train_test_split(data,target,test_size=0.2, random_state=1)
    
    print("préparation des données avec Robustscaler")
    #Standardisation des variables
    robustscaler = RobustScaler().fit(X_train)
    X_train = robustscaler.transform(X_train)
    X_test = robustscaler.transform(X_test)
    print("-->Done",end="\n\n")
    
    data=np.concatenate([X_train,X_test],axis=0)
   
    #transformation en matrice
    variables = [X_train, X_test, data, target, y_test, y_train]

    for i in range(len(variables)):
        objet = variables[i]
        if isinstance(objet, np.ndarray):
            pass  # No need to change, already a NumPy array
        else:
            variables[i] = objet.to_numpy()

    # Now variables[0], variables[1], etc. hold the NumPy arrays
    X_train, X_test, data, target, y_test, y_train = variables    
  
    return X_train,X_test,y_train,y_test,target,data,cat_indicies
    

In [32]:
def load_data2():
    #chermin vers les données
    path = r"C:\Users\Frederic\Documents\GitHub\Tennis_analysis\Prediction-of-Cirrhosis-Outcomes"
    train_path = os.path.abspath(path) + "/data" +"/train.csv"
    test_path = os.path.abspath(path) + "/data" +"/test.csv"
    
    print("creation de df_train")
    df_train = pd.read_csv(train_path,index_col='id')
    df_train  = df_train.drop(columns=["Drug"])
    
    #print(df_train.head())
    print("-->Done",end="\n\n")
    
    print("identification des variables numériques et catégorielles ")
    #identification colonnes numériques et catégorielles
    num_cols, cat_cols = find_col_dtypes(df=df_train, target='Status', limit = 4)
    print("-->Done",end="\n\n")
    
    print("la cible est la colonne :", df_train.columns.get_loc('Status'))
          
    #LabelEncoder pour les variables catégorielles
    print("Colonnes catégorielles : ",cat_cols,end="\n\n")
    
    print("Encodage des variables catégorielles avec Labelencoder")  
    le = LabelEncoder()
    for column in cat_cols:
        df_train[column] = le.fit_transform(df_train[column])
    print("-->Done",end="\n\n")

    # Séparation des fonctionnalités et de la variable cible
    y=df_train['Status']
    #Utilisation de LabelBinarizer pour encoder la variable cible 'Status'
    label_binarizer = LabelBinarizer()
    y_encoded = label_binarizer.fit_transform(y)
    df_train['Status'] = y_encoded

    print("préparation des données")
    #Préparation des données
    target = df_train.Status
    data = df_train.drop(columns=["Status"])
    print("--> Done",end="\n\n")
    
    cat_indicies = []
    for column in cat_cols:
        data[column] = data[column].astype('category')
        cat_indicies.append(data.columns.get_loc(column)) # renvoie les indices de toutes les colonnes non numériques sous forme de liste.
    print("--> les indices des colonnes catégorielles sont : ",cat_indicies)
     
    X_train,X_test,y_train,y_test = train_test_split(data,target,test_size=0.2, random_state=1)
    
    print("préparation des données avec Robustscaler")
    #Standardisation des variables
    robustscaler = RobustScaler().fit(X_train)
    X_train = robustscaler.transform(X_train)
    X_test = robustscaler.transform(X_test)
    print("-->Done",end="\n\n")
    
    data=np.concatenate([X_train,X_test],axis=0)
   
    #transformation en matrice
    variables = [X_train, X_test, data, target, y_test, y_train]

    for i in range(len(variables)):
        objet = variables[i]
        if isinstance(objet, np.ndarray):
            pass  # No need to change, already a NumPy array
        else:
            variables[i] = objet.to_numpy()

    # Now variables[0], variables[1], etc. hold the NumPy arrays
    X_train, X_test, data, target, y_test, y_train = variables    
  
    return X_train,X_test,y_train,y_test,target,data,cat_indicies
    

In [ ]:
def load_data3():
    #chermin vers les données
    path = r"C:\Users\Frederic\Documents\GitHub\Tennis_analysis\Prediction-of-Cirrhosis-Outcomes"
    train_path = os.path.abspath(path) + "/data" +"/train.csv"
    test_path = os.path.abspath(path) + "/data" +"/test.csv"
    
    print("creation de df_train")
    df_train = pd.read_csv(train_path,index_col='id')
    df_train  = df_train.drop(columns=["Drug"])
    
    #print(df_train.head())
    print("-->Done",end="\n\n")
    
    print("identification des variables numériques et catégorielles ")
    #identification colonnes numériques et catégorielles
    num_cols, cat_cols = find_col_dtypes(df=df_train, target='Status', limit = 4)
    print("-->Done",end="\n\n")
    
    print("la cible est la colonne :", df_train.columns.get_loc('Status'))
          
    #LabelEncoder pour les variables catégorielles
    print("Colonnes catégorielles : ",cat_cols,end="\n\n")
    
    print("Encodage des variables catégorielles avec Labelencoder")  
    le = LabelEncoder()
    for column in cat_cols:
        df_train[column] = le.fit_transform(df_train[column])
    print("-->Done",end="\n\n")

    # Séparation des fonctionnalités et de la variable cible
    y=df_train['Status']
    #Utilisation de LabelBinarizer pour encoder la variable cible 'Status'
    label_encoder = LabelEncoder()
    y_encoded = label_encoder.fit_transform(y)
    df_train['Status'] = y_encoded

    print("préparation des données")
    #Préparation des données
    target = df_train.Status
    data = df_train.drop(columns=["Status"])
    print("--> Done",end="\n\n")
    
    cat_indicies = []
    for column in cat_cols:
        data[column] = data[column].astype('category')
        cat_indicies.append(data.columns.get_loc(column)) # renvoie les indices de toutes les colonnes non numériques sous forme de liste.
    print("--> les indices des colonnes catégorielles sont : ",cat_indicies)
     
    X_train,X_test,y_train,y_test = train_test_split(data,target,test_size=0.2, random_state=1)
    
    print("préparation des données avec Robustscaler")
    #Standardisation des variables
    robustscaler = RobustScaler().fit(X_train)
    X_train = robustscaler.transform(X_train)
    X_test = robustscaler.transform(X_test)
    print("-->Done",end="\n\n")
    
    data=np.concatenate([X_train,X_test],axis=0)
   
    #transformation en matrice
    variables = [X_train, X_test, data, target, y_test, y_train]

    for i in range(len(variables)):
        objet = variables[i]
        if isinstance(objet, np.ndarray):
            pass  # No need to change, already a NumPy array
        else:
            variables[i] = objet.to_numpy()

    # Now variables[0], variables[1], etc. hold the NumPy arrays
    X_train, X_test, data, target, y_test, y_train = variables    
  
    return X_train,X_test,y_train,y_test,target,data,cat_indicies

In [30]:
def plot_feature_importance(importance,names,model_type):
    feature_importance = np.array(importance)
    feature_names = np.array(names)
    data={'feature_names':feature_names,
         'feature_importance':feature_importance}
    fi_df = pd.DataFrame(data)
    fi_df.sort_values(by=['feature_importance'],    
                     ascending=False,inplace=True)
    plt.figure(figsize=(10,8))
    sns.barplot(x=fi_df['feature_importance'], 
               y=fi_df['feature_names'])
    plt.title(model_type + ' FEATURE IMPORTANCE')
    plt.xlabel('FEATURE IMPORTANCE')
    plt.ylabel('FEATURE NAMES')

## Explications Encodage des variables catégorielles
"""
Label encoding is a technique where categorical data is represented as numerical values.
each unique category or label is assigned a unique integer value. Often used when working with algorithms
that require numerical input, such as decision trees and support vector machines.
                                                                    
one-hot encoding : label encoding introduces ordinal relationships between the categories, which might not be appropriate for all types of
categorical variables. For example, if you have nominal data (categories with no inherent order), using one-hot encoding might be more suitable.

Binary encoding is another technique used in data preprocessing, especially with categorical data. It transforms categorical values into
binary code, which is particularly useful when dealing with high-cardinality categorical features (features with a large number of
unique categories). Binary encoding reduces the dimensionality of the data compared to one-hot encoding, making it more memory-efficient.
"""

<img src="Encodage_variables.png" width="800" height="1200">

[Lien vers un notebook Kaggle](https://www.kaggle.com/code/markuslill/s3e26-xgbclassifer?scriptVersionId=155239835&cellId=25)